<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #7: Transfer Object Detection models to Zenodo</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso-object-detection and its requirements</font> { vertical-output: true }
from IPython.display import clear_output

try:
    import google.colab
    import os
    import sys

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/kso-object-detection.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso-object-detection/requirements.txt)
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso-object-detection/yolov5_tracker/requirements.txt)
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso-object-detection/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("kso-object-detection/tutorials")
    sys.path.append("..")

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import pkgutil

    if (
        pkgutil.find_loader("torch") is None
        or pkgutil.find_loader("torchvision") is None
    ):
        %pip install -q --upgrade pip
        %pip install -q torch torchvision

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Import specific requirements</font> { vertical-output: true }
import os
import sys

try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), ".."))) 
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set to display dataframes as interactive tables
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t6_utils as t6
import kso_utils.t7_utils as t7
import kso_utils.zenodo_utils as zenodo_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor

clear_output()
print("Packages loaded successfully")

# Find model and add to Zenodo

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose your project</font> { vertical-output: true }
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose the model</font> { vertical-output: true }
model = t6.choose_model(pp.project.Project_name)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder to download the model</font> { vertical-output: true }
download_dir = t_utils.choose_folder(".", "downloaded model")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = mlp.get_model(model.value, download_dir.selected)

# ZENODO Upload

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Provide access token to your account</font> { vertical-output: true }
# It is found in API settings by going to your Zenodo profile and clicking on Applications.
ACCESS_TOKEN = ""

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Find archive for upload to Zenodo</font> { vertical-output: true }
archive_dir = t_utils.choose_folder(".", "archive for upload")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Add archive to upload</font> { vertical-output: true }
depo_id = zenodo_utils.upload_archive(ACCESS_TOKEN, artifact_dir=artifact_dir)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Indicate database title</font> { vertical-output: true }
upload_title = t7.choose_text("title")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Write database description</font> { vertical-output: true }
upload_description = t7.choose_text("description")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Indicate database authors together with  their organisations</font> { vertical-output: true }
authors = t7.WidgetMaker()
authors

In [ ]:
authors.author_dict

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Add dataset metadata to upload</font> { vertical-output: true }
zenodo_utils.add_metadata_zenodo_upload(
    ACCESS_TOKEN,
    depo_id,
    upload_title.value,
    upload_description.value,
    authors.author_dict,
)

In [ ]:
# END